In [0]:
import os
import numpy as np

from keras.utils import to_categorical
from keras.datasets import fashion_mnist
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.models import Sequential
from keras.regularizers import l1, l2

Using TensorFlow backend.


In [0]:
!pip install -U -q PyDrive

     |████████████████████████████████| 993kB 3.4MB/s 


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

def load_from_drive(name, id):
    module = drive.CreateFile({'id': id})
    module.GetContentFile(name)

file_list = drive.ListFile({'q': "'1CBd-myDP_NKbTovfcwYsam_0H06Bl2Q_' in parents and trashed=false"}).GetList()
for file in file_list:
    if file['title'][-3:] == '.py':
        print("load file: {}, id: {}".format(file['title'], file['id']))
        load_from_drive(file['title'], file['id'])

from ReNetLayer import *
from ReduceImbalance import *
from HilbertLayer import *
from ModifiedReNetLayer import *


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

load file: ReNetLayer.py, id: 1oohHESF2OGoErRZoEftPAW3m5IXtcwGG
load file: ModifiedReNetLayer.py, id: 11ylcxUGgwS5Yzf_8zRIUzz9a_c--lznh
load file: ReduceImbalance.py, id: 1YG1ZkJfhrJeUNGGruk5iCsCKgK3ukM3H
load file: HilbertLayer.py, id: 1FuFE0OpWp0BFxNFAYGBLmdLXcHkNOSYS
load file: HilbertCurve.py, id: 1ABiyahqt1c1NkR2XIDyfvHyqHeCDkAn8
load file: CommandsGenerator.py, id: 1o4ow1uW7-8cgiAm1fwTF7xyM9piB_ZQ2
load file: ModelIfc.py, id: 1jsKb1mjhEIbAUaFW7BOZ0br8SNGXbr-p
load file: RandomSearch.py, id: 1YXiIrn0osAW4wBBAsYxwAb8BeftQmWyB
load file: KerasAdapter.py, id: 1cTv-dqemRA9j1xgd2TFVUNwtdqXqP3Hk


In [0]:
num_classes = 10
img_rows, img_cols = 28, 28
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

def normalize(matrix):
    mu = np.mean(matrix, axis=0)
    std = np.std(matrix, axis=0)
    return (matrix - mu) / std

x_train_data = normalize(x_train)
x_test_data = normalize(x_test)

print("x_train: ", x_train.shape)
print("y_train: ", y_train.shape)
print("x_test: ", x_test.shape)
print("y_test: ", y_test.shape)

print("y_train bincount: ", np.bincount(np.squeeze(y_train)))
print("y_test bincount: ", np.bincount(np.squeeze(y_test)))

# convert class vectors to binary class matrices
y_train_data = to_categorical(y_train, num_classes)
y_test_data = to_categorical(y_test, num_classes)

4423680/4422102 [==============================] - 1s 0us/step
x_train:  (60000, 28, 28, 1)
y_train:  (60000,)
x_test:  (10000, 28, 28, 1)
y_test:  (10000,)
y_train bincount:  [6000 6000 6000 6000 6000 6000 6000 6000 6000 6000]
y_test bincount:  [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]


In [0]:
from PIL import Image

def resize_data(data, new_size):
    num_samples = data.shape[0]
    resized = np.zeros((num_samples,)+new_size+(1,), dtype=data.dtype)

    for i in range(num_samples):
        img = Image.fromarray(np.squeeze(data[i]))
        resized_img = np.asarray(img.resize(new_size))
        resized[i] = np.expand_dims(resized_img, axis=3)
    
    return resized

new_size = (32, 32)
x_train_data = resize_data(x_train_data, new_size)
x_test_data = resize_data(x_test_data, new_size)
print(x_train_data.shape)
print(x_test_data.shape)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  # Remove the CWD from sys.path while we load stuff.


(60000, 32, 32, 1)
(10000, 32, 32, 1)


In [0]:
def get_reNet(lr=0.001, dense_reg=l1(0.0000001), softmax_reg=l2(0.0000001), reNet_hidden_size=256):
    model = Sequential()
    
    model.add(ReNetLayer([[2, 2]], reNet_hidden_size,
            use_dropout=True, dropout_rate=0.1))
    model.add(ReNetLayer([[2, 2]], reNet_hidden_size,
            use_dropout=True, dropout_rate=0.1))

    model.add(Flatten())
    fully_conn_hidden_size = 4096
    model.add(Dense(fully_conn_hidden_size, activation='relu', activity_regularizer=dense_reg))
    model.add(Dropout(0.1))

    num_classes = 10
    model.add(Dense(num_classes, activation='softmax', kernel_regularizer=softmax_reg))

    model.compile(loss='categorical_crossentropy',
            optimizer=Adam(lr=lr),
            metrics=['categorical_accuracy'])
    
    return model

def get_modif_reNet(lr=0.001, dense_reg=None, softmax_reg=None):
    model = Sequential()
    
    reNet_hidden_size = 256
    model.add(HilbertLayer())
    model.add(ModifiedReNetLayer(4, reNet_hidden_size,
            use_dropout=True, dropout_rate=0.1))
    model.add(ModifiedReNetLayer(4, reNet_hidden_size,
            use_dropout=True, dropout_rate=0.1))
    model.add(ModifiedReNetLayer(4, reNet_hidden_size,
            use_dropout=True, dropout_rate=0.1))
    model.add(ModifiedReNetLayer(4, reNet_hidden_size,
            use_dropout=True, dropout_rate=0.1))
    model.add(ModifiedReNetLayer(4, reNet_hidden_size,
            use_dropout=True, dropout_rate=0.1))
    

    model.add(Flatten())
    fully_conn_hidden_size = 4096
    model.add(Dense(fully_conn_hidden_size, activation='relu', activity_regularizer=dense_reg))
    model.add(Dropout(0.1))

    model.add(Dense(num_classes, activation='softmax', kernel_regularizer=softmax_reg))

    model.compile(loss='categorical_crossentropy',
            optimizer=Adam(lr=lr),
            metrics=['categorical_accuracy']
        )
    
    return model

def get_conv(lr=0.001, dense_reg=l1(0.00000001), softmax_reg=l2(0.00000001), conv_reg=l2(0.00000001)):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(4, 4), strides=(1,1), padding='same', activation='relu', kernel_regularizer=conv_reg))
    model.add(BatchNormalization())
    model.add(Conv2D(32, kernel_size=(4, 4), strides=(1,1), padding='same', activation='relu', kernel_regularizer=conv_reg))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    #model.add(Dropout(0.25))

    model.add(Conv2D(64, kernel_size=(2, 2), strides=(1,1), padding='same', activation='relu', kernel_regularizer=conv_reg))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(2, 2), strides=(1,1), padding='same', activation='relu', kernel_regularizer=conv_reg))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    #model.add(Dropout(0.25))

    model.add(Conv2D(128, kernel_size=(2, 2), strides=(1,1), padding='same', activation='relu', kernel_regularizer=conv_reg))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    #model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1024, activation='relu', activity_regularizer=dense_reg))
    model.add(Dropout(0.7))

    model.add(Dense(num_classes, activation='softmax', kernel_regularizer=softmax_reg))

    model.compile(loss='categorical_crossentropy',
            optimizer=Adam(lr=lr),
            metrics=['categorical_accuracy']
        )
    
    return model

In [0]:
model = get_reNet(lr=0.001, dense_reg=None, softmax_reg=None)

In [0]:
model = get_modif_reNet(lr=0.001, dense_reg=None, softmax_reg=None)

In [0]:
model = get_conv(lr=0.001, dense_reg=l1(0.00000001), softmax_reg=l2(0.00000001), conv_reg=l2(0.00000001))

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
x_train_single_ex = x_train_data[0:1]
y_train_single_ex = y_train_data[0:1]
model.fit(x_train_single_ex, y_train_single_ex, epochs=1)
model.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 2.0382 - categorical_accuracy: 0.0000e+00
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        544       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        16416     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 32)        128       
_

results so far for conv:
conv nets:
base confing
loss: 0.2939 - categorical_accuracy: 0.8936 - val_loss: 0.2514 - val_categorical_accuracy: 0.9087

batch norm added
test loss 0.2574437574401498
test acc:  0.9289


additional layers added
test loss 0.253200468877703
test acc:  0.9311

more neurons in last layers
test loss 0.2567021854201332
test acc:  0.9315

dropout=0.7 in last layer
test loss 0.23952612873092294
test acc:  0.9324

same as prev but with regularization: get_conv(lr=0.001, dense_reg=l1(0.00000001), softmax_reg=l2(0.00000001), conv_reg=l2(0.00000001))


In [0]:
datagen = ImageDataGenerator(width_shift_range=[-2.0, 0.0, 2.0])
datagen.fit(x_train_data)

batch_size = 32
history = model.fit_generator(datagen.flow(x_train_data, y_train_data, 
        batch_size=batch_size),
        epochs=20,
        steps_per_epoch=np.ceil(x_train.shape[0] / batch_size),
        validation_data=(x_test_data, y_test_data),
        callbacks=[EarlyStopping(monitor='val_loss', patience=20, verbose=1),
                   ReduceLROnPlateau(monitor='val_loss', patience=5, verbose=1)
            ]
    )
loss, acc = tuple(model.evaluate(x_test_data, y_test_data, batch_size=batch_size))
print("test loss", loss)
print("test acc: ", acc)

Epoch 1/20
1875/1875 [==============================] - 686s 366ms/step - loss: 0.6349 - categorical_accuracy: 0.7955 - val_loss: 0.3824 - val_categorical_accuracy: 0.8654
Epoch 2/20
1875/1875 [==============================] - 704s 375ms/step - loss: 0.4212 - categorical_accuracy: 0.8549 - val_loss: 0.3439 - val_categorical_accuracy: 0.8814
Epoch 3/20
1875/1875 [==============================] - 697s 372ms/step - loss: 0.3624 - categorical_accuracy: 0.8745 - val_loss: 0.2703 - val_categorical_accuracy: 0.8998
Epoch 4/20
1875/1875 [==============================] - 691s 368ms/step - loss: 0.3135 - categorical_accuracy: 0.8900 - val_loss: 0.2770 - val_categorical_accuracy: 0.8996
Epoch 5/20
1875/1875 [==============================] - 688s 367ms/step - loss: 0.2815 - categorical_accuracy: 0.8999 - val_loss: 0.2564 - val_categorical_accuracy: 0.9086
Epoch 6/20
1875/1875 [==============================] - 688s 367ms/step - loss: 0.2563 - categorical_accuracy: 0.9068 - val_loss: 0.2507 - v

In [0]:
x_train_single_ex = x_train_data[0:1]
y_train_single_ex = y_train_data[0:1]

for reg in [0.0000001, 0.00000001]:
    model = get_modif_reNet(lr=0.001, dense_reg=l1(reg), softmax_reg=l2(reg))
    
    model.fit(x_train_single_ex, y_train_single_ex, epochs=1)
    model.summary()
    
    datagen = ImageDataGenerator(width_shift_range=[-2.0, 0.0, 2.0])
    datagen.fit(x_train_data)
    batch_size = 32
    history = model.fit_generator(datagen.flow(x_train_data, y_train_data, 
        batch_size=batch_size),
        epochs=20,
        steps_per_epoch=np.ceil(x_train.shape[0] / batch_size),
        validation_data=(x_test_data, y_test_data),
        callbacks=[EarlyStopping(monitor='val_loss', patience=20, verbose=1),
                   ReduceLROnPlateau(monitor='val_loss', patience=5, verbose=1)])
    loss, acc = tuple(model.evaluate(x_test_data, y_test_data, batch_size=batch_size))
    print("reg = ", reg)
    print("test loss", loss)
    print("test acc: ", acc)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
1/1 [==============================] - 9s 9s/step - loss: 2.3040 - categorical_accuracy: 0.0000e+00
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hilbert_layer_1 (HilbertLaye (None, 1024, 1)           0         
_________________________________________________________________
modified_re_net_layer_1 (Mod (None, 256, 512)          0         
_________________________________________________________________
modified_re_net_layer_2 (Mod (None, 64, 512)           0         
_________________________________________________________________
modified_re_net_layer_3 (Mod (None, 16, 512)           0         
_____________________________________________________________

KeyboardInterrupt: ignored

In [0]:
import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
x_train_single_ex = x_train_data[0:1]
y_train_single_ex = y_train_data[0:1]

for hidden_size in [32, 64, 128, 256, 512, 1024, 2048, 4096]:
    print("hidden_size = ", hidden_size)
    model = get_modif_reNet(reNet_hidden_size=hidden_size)
    
    model.fit(x_train_single_ex, y_train_single_ex, epochs=1)
    model.summary()
    
    datagen = ImageDataGenerator(width_shift_range=[-2.0, 0.0, 2.0])
    datagen.fit(x_train_data)

    batch_size = 32
    model.fit_generator(datagen.flow(x_train_data, y_train_data, 
        batch_size=batch_size),
        epochs=20,
        steps_per_epoch=np.ceil(x_train.shape[0] / batch_size),
        validation_data=(x_test_data, y_test_data),
        callbacks=[EarlyStopping(monitor='val_loss', patience=20, verbose=1),
                   ReduceLROnPlateau(monitor='val_loss', patience=5, verbose=1)])
    
    print("hidden_size = ", hidden_size)
    print("test loss", loss)
    print("test acc: ", acc)

hidden_size =  32
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
1/1 [==============================] - 10s 10s/step - loss: 2.3157 - categorical_accuracy: 0.0000e+00
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
re_net_layer_1 (ReNetLayer)  (None, 16, 16, 64)        0         
_________________________________________________________________
re_net_layer_2 (ReNetLayer)  (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              16781312  
_________________________________________

# ***Cross validation 5 folds***

In [0]:
x = np.vstack([x_train_data, x_test_data])
y = np.hstack([convert_from_one_hot_to_labels(y_train_data), convert_from_one_hot_to_labels(y_test_data)])

def get_fold(fold_num):
    fold_num = fold_num-1 #indexing from 0
    x_train = x[train_indexes[fold_num]]
    y_train = y[train_indexes[fold_num]]
    x_test = x[test_indexes[fold_num]]
    y_test = y[test_indexes[fold_num]]

    x_train, y_train = undersample_to_lowest_cardinality_class(x_train, y_train)
    
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    
    return x_train, y_train, x_test, y_test

In [0]:
from sklearn.model_selection import StratifiedKFold

def get_splits(x, y):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

    train_indexes = []
    test_indexes = []
    for train_index, test_index in skf.split(x, y):
        print("train_index: ", train_index)
        print("test_index: ", test_index)
        train_indexes.append(train_index)
        test_indexes.append(test_index)
        
    return train_indexes, test_indexes

train_indexes, test_indexes = get_splits(x, y)

train_index:  [    0     1     3 ... 69996 69997 69999]
test_index:  [    2     8    12 ... 69990 69994 69998]
train_index:  [    0     1     2 ... 69995 69998 69999]
test_index:  [    6     9    13 ... 69992 69996 69997]
train_index:  [    0     1     2 ... 69997 69998 69999]
test_index:  [   10    11    19 ... 69982 69986 69993]
train_index:  [    1     2     3 ... 69997 69998 69999]
test_index:  [    0     4     7 ... 69976 69983 69995]
train_index:  [    0     2     4 ... 69996 69997 69998]
test_index:  [    1     3     5 ... 69988 69991 69999]


In [0]:
def test_model_on_fold(model, x_train, y_train, x_test, y_test):
    x_train_single_ex = x_train[0:1]
    y_train_single_ex = y_train[0:1]
    model.fit(x_train_single_ex, y_train_single_ex, epochs=1)
    
    datagen = ImageDataGenerator(width_shift_range=[-2.0, 0.0, 2.0])
    datagen.fit(x_train)

    batch_size = 32
    model.fit_generator(datagen.flow(x_train, y_train, 
            batch_size=batch_size),
            epochs=50,
            verbose=2,
            steps_per_epoch=np.ceil(x_train.shape[0] / batch_size),
            validation_data=(x_test, y_test),
            callbacks=[EarlyStopping(monitor='val_loss', patience=20, verbose=1, restore_best_weights=True),
                   ReduceLROnPlateau(monitor='val_loss', patience=5, verbose=1)
            ]
        )
    loss, acc = tuple(model.evaluate(x_test, y_test, batch_size=batch_size))
    return loss, acc
    

# ***Fold 1***

In [0]:
x_train, y_train, x_test, y_test = get_fold(1)

In [0]:
model = get_reNet()

loss, acc = test_model_on_fold(model, x_train, y_train, x_test, y_test)
print("test loss", loss)
print("test acc: ", acc)

In [0]:
model = get_modif_reNet()

loss, acc = test_model_on_fold(model, x_train, y_train, x_test, y_test)
print("test loss", loss)
print("test acc: ", acc)

In [0]:
model = get_conv()

loss, acc = test_model_on_fold(model, x_train, y_train, x_test, y_test)
print("test loss", loss)
print("test acc: ", acc)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 6.9546 - categorical_accuracy: 0.0000e+00
Epoch 1/50
 - 640s - loss: 0.6530 - categorical_accuracy: 0.7873 - val_loss: 0.4055 - val_categorical_accuracy: 0.8600
Epoch 2/50
 - 643s - loss: 0.4338 - categorical_accuracy: 0.8502 - val_loss: 0.3650 - val_categorical_accuracy: 0.8734
Epoch 3/50
 - 624s - loss: 0.3761 - categorical_accuracy: 0.8686 - val_loss: 0.3048 - val_categorical_accuracy: 0.8876
Epoch 4/50
 - 654s - loss: 0.3354 - categorical_accuracy: 0.8829 - val_loss: 0.2793 - val_categorical_accuracy: 0.9009
Epoch 5/50
 - 651s - loss: 0.3019 - categorical_accuracy: 0.8940 - val_loss: 0.2572 - val_categorical_accuracy: 0.9090
Epoch 6/50
 - 629s - loss: 0.2711 - categorical_accuracy: 0.

# ***Fold 2***

In [0]:
x_train, y_train, x_test, y_test = get_fold(2)

In [0]:
model = get_reNet()

loss, acc = test_model_on_fold(model, x_train, y_train, x_test, y_test)
print("test loss", loss)
print("test acc: ", acc)

In [0]:
model = get_modif_reNet()

loss, acc = test_model_on_fold(model, x_train, y_train, x_test, y_test)
print("test loss", loss)
print("test acc: ", acc)

In [0]:
model = get_conv()

loss, acc = test_model_on_fold(model, x_train, y_train, x_test, y_test)
print("test loss", loss)
print("test acc: ", acc)

Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 7.3309 - categorical_accuracy: 0.0000e+00
Epoch 1/50
 - 666s - loss: 0.6580 - categorical_accuracy: 0.7869 - val_loss: 0.4015 - val_categorical_accuracy: 0.8507
Epoch 2/50
 - 659s - loss: 0.4342 - categorical_accuracy: 0.8487 - val_loss: 0.3170 - val_categorical_accuracy: 0.8882
Epoch 3/50
 - 634s - loss: 0.3736 - categorical_accuracy: 0.8682 - val_loss: 0.2916 - val_categorical_accuracy: 0.8959
Epoch 4/50
 - 633s - loss: 0.3347 - categorical_accuracy: 0.8816 - val_loss: 0.2684 - val_categorical_accuracy: 0.9017
Epoch 5/50
 - 632s - loss: 0.3018 - categorical_accuracy: 0.8928 - val_loss: 0.2531 - val_categorical_accuracy: 0.9078
Epoch 6/50
 - 642s - loss: 0.2735 - categorical_accuracy: 0.9030 - val_loss: 0.2522 - val_categorical_accuracy: 0.9115
Epoch 7/50
 - 669s - loss: 0.2515 - categorical_accuracy: 0.9092 - val_loss: 0.2337 - val_categorical_accuracy: 0.9174
Epoch 8/50
 - 656s - loss: 0.2323 - categorical_accuracy:

# ***Fold 3***

In [0]:
x_train, y_train, x_test, y_test = get_fold(3)

In [0]:
model = get_reNet()

loss, acc = test_model_on_fold(model, x_train, y_train, x_test, y_test)
print("test loss", loss)
print("test acc: ", acc)

In [0]:
model = get_modif_reNet()

loss, acc = test_model_on_fold(model, x_train, y_train, x_test, y_test)
print("test loss", loss)
print("test acc: ", acc)

In [0]:
model = get_conv()

loss, acc = test_model_on_fold(model, x_train, y_train, x_test, y_test)
print("test loss", loss)
print("test acc: ", acc)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 6.4195 - categorical_accuracy: 0.0000e+00
Epoch 1/50
 - 627s - loss: 0.6463 - categorical_accuracy: 0.7895 - val_loss: 0.3841 - val_categorical_accuracy: 0.8591
Epoch 2/50
 - 613s - loss: 0.4292 - categorical_accuracy: 0.8523 - val_loss: 0.3543 - val_categorical_accuracy: 0.8763
Epoch 3/50
 - 613s - loss: 0.3754 - categorical_accuracy: 0.8694 - val_loss: 0.2948 - val_categorical_accuracy: 0.8974
Epoch 4/50
 - 616s - loss: 0.3357 - categorical_accuracy: 0.8825 - val_loss: 0.2701 - val_categorical_accuracy: 0.9034
Epoch 5/50
 - 615s - loss: 0.2983 - categorical_accuracy: 0.8942 - val_loss: 0.2433 - val_categorical_accuracy: 0.9125
Epoch 6/50
 - 611s - loss: 0.2714 - categorical_accuracy: 0.

# ***Fold 4***

In [0]:
x_train, y_train, x_test, y_test = get_fold(4)

In [0]:
model = get_reNet()

loss, acc = test_model_on_fold(model, x_train, y_train, x_test, y_test)
print("test loss", loss)
print("test acc: ", acc)

In [0]:
model = get_modif_reNet()

loss, acc = test_model_on_fold(model, x_train, y_train, x_test, y_test)
print("test loss", loss)
print("test acc: ", acc)

In [0]:
model = get_conv()

loss, acc = test_model_on_fold(model, x_train, y_train, x_test, y_test)
print("test loss", loss)
print("test acc: ", acc)

Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 1.3053 - categorical_accuracy: 1.0000
Epoch 1/50
 - 620s - loss: 0.6480 - categorical_accuracy: 0.7886 - val_loss: 0.3738 - val_categorical_accuracy: 0.8630
Epoch 2/50
 - 626s - loss: 0.4293 - categorical_accuracy: 0.8510 - val_loss: 0.3085 - val_categorical_accuracy: 0.8863
Epoch 3/50
 - 625s - loss: 0.3740 - categorical_accuracy: 0.8686 - val_loss: 0.2767 - val_categorical_accuracy: 0.8987
Epoch 4/50
 - 627s - loss: 0.3240 - categorical_accuracy: 0.8851 - val_loss: 0.2958 - val_categorical_accuracy: 0.8948
Epoch 5/50
 - 634s - loss: 0.2973 - categorical_accuracy: 0.8948 - val_loss: 0.2646 - val_categorical_accuracy: 0.9047
Epoch 6/50
 - 638s - loss: 0.2685 - categorical_accuracy: 0.9041 - val_loss: 0.2423 - val_categorical_accuracy: 0.9134
Epoch 7/50
 - 635s - loss: 0.2435 - categorical_accuracy: 0.9123 - val_loss: 0.2190 - val_categorical_accuracy: 0.9212
Epoch 8/50
 - 636s - loss: 0.2280 - categorical_accuracy: 0.9

# ***Fold 5***

In [0]:
x_train, y_train, x_test, y_test = get_fold(5)

In [0]:
model = get_reNet()

loss, acc = test_model_on_fold(model, x_train, y_train, x_test, y_test)
print("test loss", loss)
print("test acc: ", acc)

In [0]:
model = get_modif_reNet()

loss, acc = test_model_on_fold(model, x_train, y_train, x_test, y_test)
print("test loss", loss)
print("test acc: ", acc)

In [0]:
model = get_conv()

loss, acc = test_model_on_fold(model, x_train, y_train, x_test, y_test)
print("test loss", loss)
print("test acc: ", acc)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 2.8508 - categorical_accuracy: 0.0000e+00
Epoch 1/50
 - 621s - loss: 0.6617 - categorical_accuracy: 0.7859 - val_loss: 0.3668 - val_categorical_accuracy: 0.8661
Epoch 2/50
 - 621s - loss: 0.4343 - categorical_accuracy: 0.8500 - val_loss: 0.3346 - val_categorical_accuracy: 0.8772
Epoch 3/50
 - 621s - loss: 0.3722 - categorical_accuracy: 0.8688 - val_loss: 0.2923 - val_categorical_accuracy: 0.8950
Epoch 4/50
 - 616s - loss: 0.3324 - categorical_accuracy: 0.8846 - val_loss: 0.2727 - val_categorical_accuracy: 0.8996
Epoch 5/50
 - 620s - loss: 0.2971 - categorical_accuracy: 0.8944 - val_loss: 0.2646 - val_categorical_accuracy: 0.9059
Epoch 6/50
 - 622s - loss: 0.2702 - categorical_accuracy: 0.